# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_jewel_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 我对珠宝了解不多，你能给我一些建议吗？\n[销售回答] 选择珠宝主要看个人风格、场合和预算。我可以先帮您了解不同的宝石和金属类型，根据您的需求推荐一些款式。')

In [6]:
len(docs)

56

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(api_key='sk-q7U5VJZewmLYjy8HF5C6Bb74631d48C883C5C2960bD59c9c', base_url='https://api.xiaoai.plus/v1'))

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "有哪款可以作为生日礼物"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？
[销售回答] 生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。

[客户问题] 我想要一个特别的礼物给我的妻子，你有什么推荐？
[销售回答] 我们的定制系列非常受欢迎，我们可以根据她的喜好和个性，为您打造一款独一无二的珠宝。

[客户问题] 我不确定这款珠宝是否适合我。
[销售回答] 您可以试戴几件，看看哪一款最让您心动。

[客户问题] 我该如何选择送给长辈的珠宝？
[销售回答] 对于长辈，您可能需要选择一些具有经典设计和符合传统品味的款式。



In [12]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [13]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F89AF65930>, search_kwargs={'k': 3})

In [15]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？
[销售回答] 生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。

[客户问题] 我想要一个特别的礼物给我的妻子，你有什么推荐？
[销售回答] 我们的定制系列非常受欢迎，我们可以根据她的喜好和个性，为您打造一款独一无二的珠宝。

[客户问题] 我不确定这款珠宝是否适合我。
[销售回答] 您可以试戴几件，看看哪一款最让您心动。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有便宜点的项链？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我的预算有限，有哪些性价比高的选择？
[销售回答] 在您的预算范围内，我们推荐这几款戒指和项链，它与钻石外观相似，但价格更加亲民，是追求性价比客户的理想选择。

[客户问题] 这款项链适合什么场合佩戴？
[销售回答] 这款项链设计典雅而不失现代感，既适合日常佩戴，也能在正式场合下彰显您的高贵气质。

[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？
[销售回答] 生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [21]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}
)

In [22]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？
[销售回答] 生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。

[客户问题] 我想要一个特别的礼物给我的妻子，你有什么推荐？
[销售回答] 我们的定制系列非常受欢迎，我们可以根据她的喜好和个性，为您打造一款独一无二的珠宝。

[客户问题] 我不确定这款珠宝是否适合我。
[销售回答] 您可以试戴几件，看看哪一款最让您心动。

[客户问题] 我该如何选择送给长辈的珠宝？
[销售回答] 对于长辈，您可能需要选择一些具有经典设计和符合传统品味的款式。



### 提取向量数据库中的`销售回答`

In [23]:
docs = retriever.get_relevant_documents(query)

In [24]:
docs[0].page_content

'[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？\n[销售回答] 生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。'

In [25]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我想买一条项链作为生日礼物，你有什么推荐？\n',
 '生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。']

In [26]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [27]:
ans

'生日礼物选择很重要，考虑到它代表了您的心意。这条带有她出生月份宝石的项链是个不错的选择。'

#### 尝试各种问题

In [29]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [30]:
query = "可以试戴吗"

print(sales(query))

['当然可以，试戴是挑选珠宝的重要环节。', '您可以试戴几件，看看哪一款最让您心动。']


In [31]:
print(sales(query, 0.75))

['当然可以，试戴是挑选珠宝的重要环节。', '您可以试戴几件，看看哪一款最让您心动。', '我们鼓励顾客在店内试戴不同款式的珠宝，亲自体验它们的质感与适配度，以选择那件最触动您心弦的一款。', '。如果您在佩戴过程中遇到任何问题，我们随时欢迎您回来，我们将提供免费调整服务。']


In [32]:
query = "哪款比较具有性价比"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['当然有！我们有许多性价比高的选项，比如这款银饰镶嵌天然珍珠，不仅美观，而且价格适中。', '在您的预算范围内，我们推荐这几款戒指和项链，它与钻石外观相似，但价格更加亲民，是追求性价比客户的理想选择。']

score:0.5 ans: ['当然有！我们有许多性价比高的选项，比如这款银饰镶嵌天然珍珠，不仅美观，而且价格适中。', '在您的预算范围内，我们推荐这几款戒指和项链，它与钻石外观相似，但价格更加亲民，是追求性价比客户的理想选择。', '您可以试戴几件，看看哪一款最让您心动。', '这是我们的畅销款之一，它融合了经典与时尚元素，深受我们客户的喜爱。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [33]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [34]:
qa_chain({"query": "你们有200万的珍珠项链吗？"})

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们有200万的珍珠项链吗？', 'result': '对不起，我不知道。我没有关于商品的具体信息。'}

In [35]:
qa_chain({"query": "这个宝石的原产地是哪里"})

{'query': '这个宝石的原产地是哪里',
 'result': '这款宝石产自哥伦比亚，那里是世界上最著名的宝石产地之一，以其高品质和独特色泽闻名于世。'}

In [36]:
print(sales("这个宝石的原产地是哪里"))

['这款宝石产自哥伦比亚，那里是世界上最著名的宝石产地之一，以其高品质和独特色泽闻名于世。', '这颗宝石是100%天然的，我们所有的宝石都经过严格的检测和认证，确保其天然属性和品质，您可以放心购买。']


## 加载 FAISS 向量数据库已有结果

In [39]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [40]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [41]:
qa_chain({"query": "我想买玛瑙项链，你们有推荐吗"})

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买玛瑙项链，你们有推荐吗',
 'result': '对不起，作为一个人工智能，我没有能力推荐具体的产品或服务。我建议你可以在网上或实体店寻找你喜欢的玛瑙项链。'}

In [42]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [43]:
qa_chain({"query": "我想买玛瑙项链，你们有推荐吗"})

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买玛瑙项链，你们有推荐吗',
 'result': '对不起，作为一个AI，我并没有能力推荐具体的商品或服务。我建议你根据自己的喜好和需求，到信誉良好的珠宝店或在线平台上寻找你喜欢的玛瑙项链。'}

In [44]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [45]:
result = qa_chain({"query": "我想买玛瑙项链，你们有推荐吗"})

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [46]:
result

{'query': '我想买玛瑙项链，你们有推荐吗',
 'result': '对不起，作为一个人工智能，我无法提供购物推荐。我主要用来提供信息和回答问题。',
 'source_documents': []}